In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import flowermd
import gsd.hoomd
import matplotlib as plt

In [ ]:
#for use when using signac-flow
num_pps = 30
density_sim = 1.35
num_shrink_steps = 1000
chain_lengths = 20 #this defines how many monomers are in a given chain
num_timesteps = 10000
gsd_freq = 100
log_freq = 100
r_cut_value = 2.5

In [ ]:
from flowermd.library import PPS
molecules = PPS(num_mols = num_pps, lengths=chain_lengths)

In [ ]:
from flowermd.base import Pack
system = Pack(molecules=molecules, density=density_sim, packing_expand_factor=5)

In [ ]:
from flowermd.library import OPLS_AA_PPS

system.apply_forcefield(r_cut= r_cut_value, force_field=OPLS_AA_PPS(), auto_scale=True, scale_charges=True)

In [ ]:
hoomd_forces = system.hoomd_forcefield

In [ ]:
lj_force = hoomd_forces[3]

In [ ]:
from flowermd.base import Simulation

sim = Simulation.from_system(system=system, gsd_write_freq=gsd_freq, log_write_freq=log_freq)

In [ ]:
sim.run_update_volume(n_steps=num_shrink_steps, period=1, kT=1, tau_kt=1, final_box_lengths=system.target_box)

In [ ]:
import hoomd
for writer in sim.operations.writers:
    if isinstance(writer, hoomd.write.GSD):
        writer.flush()

In [ ]:
from cmeutils.visualize import FresnelGSD

sim_visualizer = FresnelGSD(gsd_file="trajectory.gsd", frame=0, view_axis=(1, 1, 1))
sim_visualizer.view()

In [ ]:
sim_visualizer.frame = -1
sim_visualizer.view()

In [ ]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])
logger.add(sim, quantities =['timestep', 'tps'])                           

In [ ]:
file = open('log.txt', mode='w', newline='\n')

In [ ]:
table_file = hoomd.write.Table(output=file,
                               trigger=hoomd.trigger.Periodic(period=500),
                               logger=logger)
sim.operations.writers.append(table_file)

In [ ]:
#adjust n_steps depending on use
#job.sp."whatever parameter from flow template's init.py"
sim.run_NVT(n_steps= num_timesteps, kT=1.0, tau_kt=0.01)

In [3]:
help(flowermd.Simulation.run_NVT)

Help on function run_NVT in module flowermd.base.simulation:

run_NVT(self, n_steps, kT, tau_kt, thermalize_particles=True, write_at_start=True)
    Run the simulation in the NVT ensemble.
    
    Parameters
    ----------
    n_steps: int, required
        Number of steps to run the simulation.
    kT: int or hoomd.variant.Ramp, required
        The temperature to use during the simulation.
    tau_kt: float, required
        Thermostat coupling period (in simulation time units).
    thermalize_particles: bool, default True
        When set to True, assigns random velocities to all particles.
    write_at_start : bool, default True
        When set to True, triggers writers that evaluate to True
        for the initial step to execute before the next simulation
        time step.



In [ ]:
!cat log.txt

In [ ]:
import numpy as np
numbers = np.loadtxt('log.txt', usecols=(1), skiprows=(1))
average_tps = np.mean(numbers)
print("Average TPS for n =",num_pps, "is", average_tps)

In [ ]:
for writer in sim.operations.writers:
    if isinstance(writer, hoomd.write.GSD):
        writer.flush()

In [ ]:
sim_visualizer.frame=-1
sim_visualizer.view()

In [ ]:
with gsd.hoomd.open("trajectory.gsd") as traj:
    frame = traj[0] # traj is a list of snapshots, so you can access them using regular indexing
    particle_types = frame.particles.types
    print(particle_types)